This notebook will create the config file and the ner model for the food
we will add this model pipeline with existing spacy pipeline in order to make a final model which should theoretically be able to identify
the food items and the amounts correctly

python -m spacy init fill-config base_config.cfg config.cfg

python -m spacy train SpaCy/TRAIN/configGPU.cfg --output SpaCy/model/  --paths.train datasets/spacyFiles/TrainData.spacy --paths.dev datasets/spacyFIles/Dev.spacy --gpu-id 0


python -m spacy evaluate Spacy/model [test.spacy path] --output training/metrics.json

In [ ]:
import spacy
from spacy.cli.train import train
from spacy.cli.evaluate import evaluate
spacy.prefer_gpu()

In [8]:
train("./configGPU.cfg",output_path="../../SpaCy/model/",use_gpu= 0 , overrides={"paths.train": "../../datasets/spacyFiles/Train.spacy", "paths.dev": "../../datasets/spacyFiles/Dev.spacy" })

ℹ Saving to output directory: ..\..\SpaCy\model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        6485.89    322.09    0.00    0.00    0.00    0.00
 10     200      403244.54  44097.14   85.19   84.49   85.89    0.85
 20     400         896.68   2805.70   88.09   90.39   85.89    0.88
 31     600         181.80   1751.54   89.96   87.16   92.95    0.90
 41     800          99.68   1612.42   88.75   87.50   90.04    0.89
 51    1000          69.04   1502.32   88.10   88.66   87.55    0.88
 61    1200          54.48   1458.46   87.10   84.71   89.63    0.87
 71    1400          23.38   1413.51   89.16   87.90   90.46    0.89
 82    1600          34.05   1375.05   88.80   85.71   92.12    0.89
 92    1800          40.02   1370.63   88.75   87.50   90.04    0.89
102    20

In [9]:
# this will just evaluate the model with only the FOOD ent ... i think
evaluate("../../SpaCy/model/model-best/", "../../datasets/spacyFiles/Test.spacy","../../SpaCy/model/evaluationResult/metrics.json")

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.8844621513944223,
 'ents_r': 0.9367088607594937,
 'ents_f': 0.9098360655737705,
 'ents_per_type': {'FOOD': {'p': 0.8844621513944223,
   'r': 0.9367088607594937,
   'f': 0.9098360655737705}},
 'speed': 9472.163678999987}

In [10]:
# https://spacy.io/api/language#replace_listeners
nlp = spacy.load("en_core_web_trf",disable=["tagger","parser", "attribute_ruler", "lemmatizer"])  #The TRF model has a ner accuracy of 0.898 
food_nlp = spacy.load("C:\\Users\\samad\\Documents\\DISSERTATION\\SpaCy\\model\\model-best")
food_nlp.replace_listeners("transformer", "ner", ["model.tok2vec"])

nlp.add_pipe(
    "ner",
    name="ner_food",
    source=food_nlp,
    after="ner",
)

nlp.to_disk("../model/combined/")

The containers below were using for testing purposes : IGNORE



    cant change the token in a span so we just gonna convert it and save to array
    so a cardinal if its like "THREE" use numerizer() to get numerical val and use that instead

    so that the ents list is only cardinal and FOOD
    and say 
    if the item that we picked is a cardinal
        if the next ents is food then we use numerize on the cardinal
         [FOOD, cardinal._.numerize()]
    if the item picked is FOOD and next is also FOOD
        [FOOD,1]

    #if we are in a number and the next is food
    #logically thinking the number is quantity for the food 
    # while this is usual case, ofc this code can be rendered useless easily
    # because someone can say "i ate chicken wings, 20 of them"
    # and now this code will not work 😕
    
    #maybe skipping the next ent if the current is cardinal
    # eg cardinal food food food cardinal food
    # i = 0 we are at cardinal and next is food so we save [food,numerizedcardinal], then i+= 1 and also the i+=1 at the end of the while funct
    # i = 2 its food  so just i+=1
    # i = 3 its food again so just i+=1
    # i = 4 its cardinal and next food 

In [12]:

# def createList(doc,arr):
#     i = 0 
#     lenEnts = len(doc.ents)
#     while i < lenEnts:
#         currEnt = doc.ents[i] # the current ent 
#         print(i , currEnt.label_)
#         try: 
#             nextEnt = doc.ents[i+1] # the next ent in order to check
#             print(i,nextEnt.label_)
#             if currEnt.label_ == "CARDINAL" and nextEnt.label_ == "FOOD":
#                 i+=1
#                 arr.append([nextEnt.text,currEnt._.numerize()])
#             else:
#                 if (currEnt.label_ == "FOOD"): 
#                     arr.append([currEnt.text,1])
#         except: 
#             # print("no next ent at"+str(i))
#             if(currEnt.label_ == "FOOD"):
#                 arr.append([currEnt.text,1])
#             pass
#         i+=1
#     return arr

In [13]:
# from spacy import load,displacy
# import numerizer
# nlp = spacy.load("./../model/combined/")
# doc = nlp("I ate three cereal bar, half of an apple and a banana for breakfast and a ham sandwich for lunch and chicken and rice for dinner and 3 chicken thighs and chicken")

# # doc = nlp("800 grams of Locatelli's  ribeye steak")

# displacy.render(doc,style="ent",jupyter=True)
# arr = []

# #if the ent list is larger than 1 then we can do the 
# if len(doc.ents) == 1 and doc.ents[0].label_ == 'FOOD':
#     arr.append([doc.ents[0].text , 1])
# else:
#     arr = createList(doc,arr)

# print(arr)
# print(doc.ents)



0 CARDINAL
0 FOOD
2 CARDINAL
2 FOOD
4 FOOD
4 FOOD
5 FOOD
5 FOOD
6 FOOD
6 CARDINAL
7 CARDINAL
7 FOOD
9 FOOD
[['cereal bar', '3'], ['apple', '1/2'], ['banana', 1], ['ham sandwich', 1], ['chicken and rice', 1], ['chicken thighs', '3'], ['chicken', 1]]
(three, cereal bar, half, apple, banana, ham sandwich, chicken and rice, 3, chicken thighs, chicken)


In [ ]:
# from itertools import *

# doc = nlp("I ate three cereal bar, half of an apple and 500 g of ribeye steak and a ham sandwich for lunch and chicken and rice for dinner and 3 chicken thighs and chicken")
# # e = {key:list(g) for key, g in groupby(doc.ents,lambda x:x.label_)}

# displacy.render(doc,style="ent",jupyter=True)
# entities = {key: list(g) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)} 

# print(entities)